In [3]:
import nbimporter
%run ~\MSAnalysis\programs\readin.ipynb
from scipy.stats import ttest_ind as ttest
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [10]:
def n_thresholds(alist, percents=[95]):
    alist = sorted(alist, reverse=True)
    with_zeros = {}
    for i in percents:
        p = (100.0-float(i))/100.0
        t = float(alist[math.ceil(float(len(alist))*p)])
        with_zeros[i] = t
        
        print("{0}% threshold: {1}".format(i, t))

    print("\nIgnoring Zeros: ")
    alist = [x for x in alist if (x!=0)]
    skip_zeros = {}
    for i in percents:
        p = (100.0-float(i))/100.0
        t = float(alist[math.ceil(float(len(alist))*p)])
        skip_zeros[i] = t
        
        print("{0}% threshold: {1}".format(i, t))
    
    r = {
        'with_zeros':with_zeros,
        'skip_zeros':skip_zeros
    }
    
    return r

### Readin Functions -- just getting the data

In [2]:
def get_data(FILE = None, logged = None):

    if FILE == None: FILE = input("Filename: ")
    if logged == None: 
        logged = input("Press 0 to use original values\nPress 1 to use log-normalized values\nPress 2 to graph both before proceeding")

    try: logged = (int(logged))
    except: logged = 2

    print ("Reading data from", FILE)
    def ms3plot_all(data, title="All Non-Zero Data Points"):
        plt.figure(figsize=(15,7))
        plt.hist(skipZero(data.values.flatten()), alpha=.5, bins = 100)
        sns.rugplot(skipZero(data.values.flatten()), color="black")
        plt.title("All Non-Zero Data Points")
        plt.show()
        print ("Note that %i zeros are not shown, out of a dataset of %i." % 
               (len([z for z in data.values.flatten() if (z==0)]), len(data.values.flatten())))

    if logged == 0: 
        ms3data = readin(FILE)
        ms3plot_all()
    elif logged == 1: 
        ms3data = readin_log(FILE)
        ms3plot_all(title="All Non-Zero Data Points (Log-Normalized)")
    else:
        ms3data_orig =  readin(FILE)
        ms3plot_all(ms3data_orig)
        ms3data_log = readin_log(FILE)
        ms3plot_all(ms3data_log, title="All Non-Zero Data Points (Log-Normalized)")
        chosen=False
        while not chosen:
            logged = input("\nPress 0 to use original values\nPress 1 to use log-normalized values\n")
            try: logged = (int(logged))
            except: chosen=False
            if logged == 0 or logged == 1: chosen=True

    print("Done")
    return ms3data

    

In [3]:
def show_by_columns(ms3data):
    means_of_nonZero_by_col = ms3data.apply(skipZeroMean, axis='index')
    plt.hist(means_of_nonZero_by_col, bins=40)
    plt.title("Means of Columns (Log-Normalized)")
    plt.show()
    print(means_of_nonZero_by_col)

In [4]:
def show_by_proteins(ms3data):
    means_of_nonZero_by_row = ms3data.apply(skipZeroMean, axis='columns')
    plt.figure(figsize=(10,7))
    plt.hist(skipZero(means_of_nonZero_by_row), bins=500)
    plt.title("Means by Protein (Log-Normalized)")
    plt.show()


    print ("Note that %i zeros are not shown, out of a dataset of %i." % 
           (len([z for z in means_of_nonZero_by_row if (z==0)]), len(means_of_nonZero_by_row)))

### Finding Technical Replicates functions

In [5]:
def compare(selfseries, otherseries):
    c_dist = 0.0 #cumulative differences between the two series
    for i in range(0,len(selfseries)):
        c_dist += abs(float(selfseries[i])-float(otherseries[i]))
    ave_dist = float(c_dist)/float((len(selfseries)))
    return ave_dist

In [6]:
def get_lower(series, to_show=5):
    orser = series.sort_values(inplace=False)
    print (orser.iloc[range(0,to_show)], '\n\n')

In [7]:
def dif(ms3data, start = 0, stop = 10, start_2 = None, stop_2=None, to_show = 5):
    if start_2 == None: start_2 = start
    if stop_2 == None: stop_2 = stop
        
    differences = {}
    for i in range(start,stop): #The data is four sets of ten columns
        diff_c = {} #differences for this column
        selfser = ms3data.iloc[:,i]
        for o in range(start_2, stop_2):
            oser = ms3data.iloc[:,o]
            dif = compare(selfser, oser)
            diff_c[o] = dif
        differences[i] = diff_c
        
    differences = pd.DataFrame.from_dict(differences, dtype = float)
    
    v = (differences.apply(get_lower, to_show=to_show))
    
    return differences   

In [8]:
def graph_tech_difs(differences, technical_replicates):
    #technical_replicates = ([0,1,2],[3,4,5],[6,8]) #MS3
    #technical_replicates = ([10,11,12],[13,14,15],[16,18]) #MS3 uncorrected
    #technical_replicates = ([0,1],[2,3], [8,9])  #As best I can figure for Slavov as 10plex, logged var ~0.1
    #technical_replicates = ([0,1,2], [3,4,5]) #This is better => loged variability ~0.1

    tech_dif = []
    for sample in technical_replicates:
        sample_dif = []
        for i in sample:
            for x in sample:
                if i != x:
                    sample_dif.append(differences[i][x])
        sample_dif =skipZeroMean(sample_dif)
        tech_dif.append(sample_dif)

    plt.hist(tech_dif, alpha=.3, bins = 20)
    plt.title("Technical Differences")
    sns.rugplot(tech_dif)
    plt.show()
    print(tech_dif)

### Technical Variance Stats

"""technical_replicates = {
    
    "Cell Line 1":[0,1,2],
    "Cell Line 2":[3,4,5],
    "Cell Line 3":[6,8],
} #""" #MS3

In [9]:
def tech_variance_stats(ms3data, technical_replicates):
    technical_variances = []
    for sample in technical_replicates:
        #
        reps = {}
        for rep in technical_replicates[sample]:
            reps[ms3data.iloc[:,rep].name] = ms3data.iloc[:,rep]

        for r in range(0, len(list(reps.values())[0])):
            rep_for_protein = []
            for rep in reps.values():
                if rep[r] > 0: rep_for_protein.append(rep[r])

            if len(rep_for_protein) > 1: 
                v = variance(rep_for_protein)
                technical_variances.append(v)
            else: 
                technical_variances.append(0)

    fig = plt.figure(figsize=(12,8))

    plt.hist(skipZero(technical_variances), bins = 100)
    #sns.rugplot(technical_variances, color="gray")
    plt.title("Technical Variances")

    plt.rc('axes', titlesize=30)
    plt.rc('axes', labelsize=25)
    plt.rc('xtick', labelsize=15)
    plt.rc('ytick', labelsize=20) 

    plt.axvline(x= threshold, linestyle='dashed')

    plt.xlabel("Variance Within Cell Line")
    plt.ylabel("Number of Proteins")

    #fig.savefig("figures/{0}.png".format(name), dpi=300)

    plt.show()

    print ("Note that %i zeros are not shown, out of a dataset of %i." % 
           (len([z for z in technical_variances if (z==0)]), len(technical_variances)))

    technical_variances = sorted(technical_variances, reverse=True)
    print ("Technical Variances Calculated:",len(technical_variances))
    print ("Greatest Technical Variance:", technical_variances[0])

    threshold = get_thresholds(technical_variances)
    return threshold


### Variance Graphs

In [10]:
def by_sample(ms3data, technical_replicates):
    msSamples = {}
    for sample in technical_replicates:
        reps = {}
        for rep in technical_replicates[sample]:
            reps[ms3data.iloc[:,rep].name] = ms3data.iloc[:,rep]
        msSamples[sample] = pd.DataFrame.from_dict(reps, dtype = float)
    
    return msSamples

In [11]:
def skipZeroMin(alist):
    alist = skipZero(alist)
    return min(alist)

def get_least_value(_series):
    mins = []
    for l in _series:
        l_mins = _series[l].apply(skipZeroMin).tolist()
        for i in l_mins: mins.append(i)
        return min(mins)

In [12]:
def cal_fold_changes_and_variances(ms3Samples, approxZero=None):
    variances = {}
    fold_changes = {}
    
    if approxZero == None: localZero = True
    else: localZero = False
        
    sample_names = list(ms3Samples.keys())
    
    for ser_index in range(0,len(sample_names)-1): #these are the keys to ms3Samples
        ser = sample_names[ser_index]
        sample_df1 = ms3Samples[ser]
        for o_ser_index in range(ser_index+1,len(sample_names)): #keys again
            variances_averaged = {} #used in graph to show v1 vs v2
            o_ser = sample_names[o_ser_index]
            #compare variance in sample versus otherSample
            sample_df2 = ms3Samples[o_ser]
            if localZero: 
                approxZero = get_least_value({ ser:sample_df1,o_ser:sample_df2 }) / 2.0
            for protein in sample_df1.index:
                t1 = [x for x in sample_df1.loc[protein,:] if x != 0]
                t2 = [x for x in sample_df2.loc[protein,:] if x != 0]

                if len(t1) > 1: v1 = variance(t1)
                else: v1 = 0
                if len(t2) > 1: v2 = variance(t2)
                else: v2 = 0
                v = (v1+v2)/2
                variances_averaged[v1]=v2
                variances[((ser,o_ser),protein)] = v
                #this needs to implement the half-least-psuedo-zero to avoid divided by zero errors
                m2 =skipZeroMean(t2); m1 = skipZeroMean(t1)
                if m1 ==0: m1 = approxZero
                if m2 ==0: m2 = approxZero
                fold_changes[((ser,o_ser),protein)] = m2/m1

            #plt.scatter(variances_averaged.keys(), variances_averaged.values())
        #plt.show
    returnValues = {
        "fold_changes":fold_changes,
        "variances":variances
    }
        
    return returnValues

In [13]:
def graph_by_variance(ms3data, technical_replicates, FOLD_CHANGE_THRESHOLD = 2, approxZero = None, threshold95 = None,
                      threshold99 = None):
    ms3Samples = by_sample(ms3data, technical_replicates)
    
    sample_names = list(technical_replicates.keys())
    
    if approxZero == None: 
        approxZero = get_least_value(ms3Samples) / 2.0
        
    fc_v = cal_fold_changes_and_variances(ms3Samples, approxZero)
    fold_changes= fc_v["fold_changes"]
    variances = fc_v["variances"]
    
    if threshold95 ==None:
        variances_sorted = sorted(variances.values(), reverse=True)
        threshold95 = float(variances_sorted[math.ceil(float(len(variances_sorted))*.05)])
        print (threshold95)
    
    #Volcano Graph of all the data
    plt.title("Log 2 Fold Changes")
    log2_fold_changes = [math.log2(x) for x in fold_changes.values()]
    plt.scatter(log2_fold_changes, variances.values())
    plt.axvline(x=math.log2(FOLD_CHANGE_THRESHOLD), linestyle='dashed')
    plt.axvline(x=-math.log2(FOLD_CHANGE_THRESHOLD), linestyle='dashed')
    plt.axhline(y=threshold95, color='b', linestyle='-')
    if threshold99 != None:
        plt.axhline(y=threshold99, color='b', linestyle='-',alpha=.5)
    plt.gca().invert_yaxis()
    plt.show()


In [14]:
def graph_all_var(fold_changes, variances, sample_names, FOLD_CHANGE_THRESHOLD=2,
              zoomin = False, REPORT_ALL = False, threshold95=None, localThresh=False,
              skipZeroInVar=False
             ):
    
    if threshold95 == None and localThresh==False:
        variances_sorted = sorted(variances.values(), reverse=True)
        variances_sorted = skipZero(variances_sorted)
        threshold95 = float(variances_sorted[math.ceil(float(len(variances_sorted))*.05)])
        
        
    for ser_index in range(0,len(sample_names)-1): 
        ser = sample_names[ser_index]
        for o_ser_index in range(ser_index+1,len(sample_names)):
            print()
            o_ser = sample_names[o_ser_index]
            
            if localThresh:
                variances_sorted = sorted([variances[x] for x in variances if x[0] == (ser,o_ser)], reverse=True)
                threshold95 = float(variances_sorted[math.ceil(float(len(variances_sorted))*.05)])
        
            svo_fc = {x:fold_changes[x] for x in fold_changes if x[0] == (ser,o_ser)}
            svo_var = {x:variances[x] for x in variances if x[0] == (ser,o_ser)}

            log2_fold_changes = [math.log2(x) for x in svo_fc.values()]
    
            plt.rc('axes', titlesize=55)
            plt.rc('axes', labelsize=30)
            plt.rc('xtick', labelsize=25)
            plt.rc('ytick', labelsize=25) 
            
            fig, ax = plt.subplots() 
            fig.set_figheight(16)
            fig.set_figwidth(24)

            plt.title(ser+" vs "+o_ser)
            plt.scatter(log2_fold_changes, svo_var.values())

            plt.axvline(x=  math.log2(FOLD_CHANGE_THRESHOLD), linestyle='dashed')
            plt.axvline(x= -math.log2(FOLD_CHANGE_THRESHOLD), linestyle='dashed')
            plt.axhline(y=threshold95, color='b', linestyle='-')
            plt.gca().invert_yaxis()
            plt.xlabel("Log2 Fold Change")
            plt.ylabel("Technical Variance")

            if zoomin:
                axins = zoomed_inset_axes(ax, 2, loc=3) 
                axins.scatter(log2_fold_changes, svo_var.values())
                axins.set_xlim(-2.25, -1.5) # apply the x-limits
                axins.set_ylim(-0.02, 0.2)
                axins.xaxis.set_visible(False)
                axins.yaxis.set_visible(False)
                axins.invert_yaxis()
                mark_inset(ax, axins, loc1=1, loc2=2, fc="none", ec="0.5")

            fig_name = str('figures/'+ser+' vs '+o_ser+'.jpeg')
            #fig.savefig(fig_name, dpi=300)
            plt.show()

            weirdpoints = []
            up_reg_points = []
            down_reg_points = []
            for key in fold_changes:
                if variances[key] < threshold95:
                    if key[0] == ((ser,o_ser)):
                        if fold_changes[key] > FOLD_CHANGE_THRESHOLD:
                            weirdpoints.append(key)
                            up_reg_points.append(key)
                        elif fold_changes[key] < 1/FOLD_CHANGE_THRESHOLD:
                            weirdpoints.append(key)
                            down_reg_points.append(key)

            line_category_counts = {}#Example key = "Cell Line1 vs Cell line 2" value = number of occurances
            protein_category_counts = {}#Name:Frequency
            for key in weirdpoints:
                line_category = (key[0][0]+" vs "+key[0][1])
                if line_category in line_category_counts:
                    line_category_counts[line_category] += 1
                else: line_category_counts[line_category] = 1

                protein = key[1]
                if protein in protein_category_counts:
                    protein_category_counts[protein] += 1
                else:protein_category_counts[protein] = 1

            print ("{0} proteins change significantly, out of {1} ({2:.2f}%)"
                   .format(len(protein_category_counts), ms3data.shape[0], 100*len(protein_category_counts)/ms3data.shape[0]))

            print ("{0} proteins are upregulated, out of {1} ({2:.2f}%)"
                   .format(len(up_reg_points), ms3data.shape[0], len(up_reg_points)/ms3data.shape[0]*100))
            print ("{0} proteins are downregulated, out of {1} ({2:.2f}%)"
                   .format(len(down_reg_points), ms3data.shape[0], len(down_reg_points)/ms3data.shape[0]*100))
            print ("\n95% Variance Threshold:",threshold95)
            
            if REPORT_ALL==True:
                print("\nProtein\t\t  Fold Change\t\t Variance\n")
                for key in weirdpoints:
                    print (key[1],"\t{0:.4f}".format(math.log2(fold_changes[key])),
                           "\t{0:.4f}".format(variances[key]))

### T Test

In [38]:
def get_t_stats(ms3Samples, report_cat = False, approxZero = None, min_threshold = 0.0):
    t_stats = {}
    fold_changes = {}
    
    if approxZero == None: 
        approxZero = get_least_value(ms3Samples) / 2.0
        
    sample_names = list(ms3Samples.keys())
    c12, c22, c11, c21, c10,c20 = 0,0,0,0,0,0

    for ser_index in range(0,len(sample_names)-1): #these are the keys to ms3Samples
        ser = sample_names[ser_index]
        sample_df1 = ms3Samples[ser]
        for o_ser_index in range(ser_index+1,len(sample_names)): #keys again
            o_ser = sample_names[o_ser_index]
            #compare variance in sample versus otherSample
            sample_df2 = ms3Samples[o_ser]
            for protein in sample_df1.index:
                tl1 = sample_df1.loc[protein,:]
                tl2 = sample_df2.loc[protein,:]

                # if the full set is zero
                if len([x for x in tl1 if x > min_threshold]) == 0: 
                    c10 +=1
                    tl1 = [approxZero,0]
                if len([x for x in tl2 if x > min_threshold]) == 0: 
                    c20 +=1
                    tl2 = [approxZero,0]


                #if one non-null - 
                #A. leave those alone it'll be basically zero anyway
                if (len([x for x in tl1 if x > min_threshold])) ==1: c11+=1
                if (len([x for x in tl2 if x > min_threshold])) ==1: c21+=1
                #or B. toss it?
                if (len([x for x in tl1 if x > min_threshold])) ==1: tl1 = [approxZero,0]
                if (len([x for x in tl2 if x > min_threshold])) ==1: tl2 = [approxZero,0]


                #if two good and one ~null, use only two good
                if (len([x for x in tl1 if x > min_threshold])) ==2:
                    tl1 = [x for x in tl1 if x > min_threshold]
                    c12+=1
                if (len([x for x in tl2 if x > min_threshold])) ==2:
                    tl2 = [x for x in tl2 if x > min_threshold]
                    c22+=1


                m1 = mean(tl1)
                m2 = mean(tl2)

                fold_changes[((ser,o_ser),protein)] = m2/m1


                s =ttest(tl1,tl2)
                t_stats[((ser,o_ser),protein)] = s
                
    if report_cat and len(technical_replicates) == 2:
        print ("No good values, line 1:",c10)
        print ("One good value, line 1:",c11)
        print ("Two good values, line 1",c12)
        print ()
        print ("No good values, line 2:",c20)
        print ("One good value, line 2:",c21)
        print ("Two good values, line 2",c22)
        
        c = 0
        for p in t_stats.values():
            i = p[1]
            if not (i < 0) and not (i == 0) and not (i > 0):
                c += 1
        print (c,'NaN out of',(len(t_stats.values())))

    returnValues = {
        "fold_changes":fold_changes,
        "t_stats":t_stats
    }
        
    return returnValues

In [16]:
def volcano(fold_changes, t_stats, FOLD_CHANGE_THRESHOLD=2,P_VAL=.05,P_VAL2=None):
    plt.title("Log 2 Fold Changes")
    log2_fold_changes = [x if x == 0 else math.log2(x) for x in fold_changes.values()]
    p_values = [x[1] for x in t_stats.values()]
    plt.scatter(log2_fold_changes, p_values)
    plt.axvline(x= math.log2(FOLD_CHANGE_THRESHOLD), linestyle='dashed')
    plt.axvline(x=-math.log2(FOLD_CHANGE_THRESHOLD), linestyle='dashed')
    plt.axhline(y=P_VAL, color='b', linestyle='-')
    if P_VAL2: plt.axhline(y=P_VAL2, color='b', linestyle='-',alpha=.3)
    plt.gca().invert_yaxis()
    plt.show()


In [35]:
def graph_all_volcanoes(fold_changes, t_stats, sample_names, fold_change_threshold=2,
                        REPORT_ALL = False, P_VAL=.05, SAVE=False, #neg_control=None, 
                        approxZero=None, 
             ):
    #if neg_control != None:
        
    if approxZero == None: 
        approxZero = get_least_value(ms3Samples) / 2.0
        
    
    for ser_index in range(0,len(sample_names)-1): 
            ser = sample_names[ser_index]
            for o_ser_index in range(ser_index+1,len(sample_names)):
                print()
                o_ser = sample_names[o_ser_index]
                svo_fc = {x:fold_changes[x] for x in fold_changes if x[0] == (ser,o_ser)}
                svo_var = {x:t_stats[x][1] for x in t_stats if x[0] == (ser,o_ser)}

                log2_fold_changes = [x if x == 0 else math.log2(x) for x in svo_fc.values()]
                #fig = plt.figure(figsize=(12,8))
                
                plt.rc('axes', titlesize=55)
                plt.rc('axes', labelsize=30)
                plt.rc('xtick', labelsize=25)
                plt.rc('ytick', labelsize=25) 
                
                fig, ax = plt.subplots() 
                fig.set_figheight(16)
                fig.set_figwidth(24)

                plt.title(ser+" vs "+o_ser)
                plt.scatter(log2_fold_changes, svo_var.values())

                plt.axvline(x=  math.log2(fold_change_threshold), linestyle='dashed')
                plt.axvline(x= -math.log2(fold_change_threshold), linestyle='dashed')
                plt.axhline(y=P_VAL, color='b', linestyle='-')
                plt.gca().invert_yaxis()

                plt.xlabel("Log2 Fold Change")
                plt.ylabel("P Value")
                
                fig_name = str('figures/P_FC'+ser+' vs '+o_ser+'.jpeg')
                if SAVE==True: fig.savefig(fig_name, dpi=300)
                plt.show()

                weirdpoints = []
                up_reg_points = []
                down_reg_points = []
                for key in fold_changes:
                    if t_stats[key][1] < P_VAL:
                        if key[0] == ((ser,o_ser)):
                            if fold_changes[key] > fold_change_threshold:
                                weirdpoints.append(key)
                                up_reg_points.append(key)
                            elif fold_changes[key] < 1/fold_change_threshold:
                                weirdpoints.append(key)
                                down_reg_points.append(key)

                line_category_counts = {}#Example key = "Cell Line1 vs Cell line 2" value = number of occurances
                protein_category_counts = {}#Name:Frequency
                for key in weirdpoints:
                    line_category = (key[0][0]+" vs "+key[0][1])
                    if line_category in line_category_counts:
                        line_category_counts[line_category] += 1
                    else: line_category_counts[line_category] = 1

                    protein = key[1]
                    if protein in protein_category_counts:
                        protein_category_counts[protein] += 1
                    else:protein_category_counts[protein] = 1


                print ("Using a {0:.3f} fold change threshold".format(fold_change_threshold))

                print ("{0} proteins change significantly, out of {1} ({2:.2f}%)"
                       .format(len(protein_category_counts), len(fold_changes), 100*len(protein_category_counts)/len(fold_changes)))

                print ("{0} proteins are upregulated, out of {1} ({2:.2f}%)"
                       .format(len(up_reg_points), len(fold_changes), len(up_reg_points)/len(fold_changes)*100))
                print ("{0} proteins are downregulated, out of {1} ({2:.2f}%)"
                       .format(len(down_reg_points), len(fold_changes), len(down_reg_points)/len(fold_changes)*100))

                if REPORT_ALL==True:
                    print("\nProtein\t\t  Fold Change\t\t Variance\n")
                    for key in weirdpoints:
                        print (key[1],"\t{0:.4f}".format(math.log2(fold_changes[key])),
                               "\t{0:.4f}".format(variances[key]))

### Standard deviations 

In [18]:
def get_tech_st_devs(ms3data, technical_replicates):
    technical_standard_deviations = []
    for sample in technical_replicates:
        reps = {}
        for rep in technical_replicates[sample]:
            reps[ms3data.iloc[:,rep].name] = ms3data.iloc[:,rep]

        #for each protein...
        for r in range(0, len(list(reps.values())[0])):
            rep_for_protein = []
            p = ms3data.iloc[r].name
            for rep in reps.values():
                if rep[r] > 0: rep_for_protein.append(rep[r])
            if len(rep_for_protein) > 1: 
                v = stdev(rep_for_protein)
                technical_standard_deviations.append((sample, p, v))
            else: 
                technical_standard_deviations.append((sample, p, 0))
    return technical_standard_deviations

In [19]:
def calc_fold_threshold(ms3data, technical_replicates,
                        technical_standard_deviations = None,
                       percent_cuttoff = 99.0): #will be calculated if not passed in
    f = (100.0-float(percent_cuttoff))/100.0
    
    if technical_standard_deviations == None:
        technical_standard_deviations = get_tech_st_devs(ms3data, technical_replicates)
    
    technical_standard_deviations.sort(key=operator.itemgetter(2),reverse=True)
    thresh_item = technical_standard_deviations[math.ceil(float(len(technical_standard_deviations))*f)]


    rep_for_protein = []
    for rep in technical_replicates[thresh_item[0]]:
        rep_for_protein.append(ms3data.iloc[:,rep][thresh_item[1]])

    rep_min = min(rep_for_protein)
    rep_max = max(rep_for_protein)
    return ((rep_max)/rep_min)

### Negative Control

In [32]:
def eval_neg_cont(file, column):
    return_data = {}
    
    ms3data = readin(file)
    negCont = ms3data.iloc[:,column].values
    approxZeroRaw = mean(negCont)
    
    stdevBlankRaw = stdev(negCont)
    min_thresholdRaw = stdevBlankRaw+approxZeroRaw
    return_data['ave_zero_raw'] = approxZeroRaw
    return_data['stdev_zero_raw'] = stdevBlankRaw
    return_data['min_threshold_raw'] = min_thresholdRaw
    
    
    return_data['log_ave_zero_raw'] = math.log(approxZeroRaw)
    return_data['log_stdev_zero_raw'] = math.log(stdevBlankRaw)
    return_data['log_min_threshold_raw'] = math.log(min_thresholdRaw)

    ms3data = (np.log(ms3data)).replace(-np.inf, 0)
    negCont = ms3data.iloc[:,7].values
    min_threshold = math.log(min_thresholdRaw)

    return_data['ave_zero_log'] = mean(negCont)
    return_data['stdev_zero_log'] = (stdev(negCont))
    return_data['min_threshold_by_log_values'] = mean(negCont)+(stdev(negCont))
    
    return_data['zeros']:len(skipZero(neg_cont))
    return_data['zeros_fraction']:len(skipZero(neg_cont))/len(neg_cont)
    
    return return_data

In [21]:
def neg_above_thresholds(neg_cont, samples, print_all=False):
    def above(per, neg_cont, samples):
        threshold = float(samples[math.ceil(float(len(samples))*per)])
        count = len([x for x in neg_cont if x > threshold])
        if print_all:
            print ("Above {0}%".format(per*100.0), count, count/len(neg_cont))
            print (per, threshold)
        return count

    blanks_above = {'At Zero': (len(neg_cont)-len(skipZero(neg_cont)))}
    for i in range(0,50):
        p = (float(i)/100.0)
        blanks_above[p] = above(p, neg_cont, samples)

    plt.rc('axes', titlesize=30)
    plt.rc('axes', labelsize=25)
    plt.rc('xtick', labelsize=15)
    plt.rc('ytick', labelsize=20) 

    fig = plt.figure(figsize=(20,5))
    plt.bar(range(len(blanks_above)), list(blanks_above.values()), align='center')
    plt.xticks(range(len(blanks_above)), list(blanks_above.keys()), rotation="vertical")


    plt.axhline(y=(len(neg_cont)/100), color='b', linestyle='-')
    plt.axhline(y=(len(neg_cont)/20), color='b', linestyle='-',alpha=.5)

    plt.xlabel("Non-Zero Sample Data Threshold")
    plt.ylabel("Number of Proteins")

    plt.title("Neg Control Proteins Above Thresholds")

    #fig.savefig('figures/NegAboveThresholds.png', dpi=300)
    plt.show()

In [2]:
def graph_neg_vs_samples(samples, neg_cont, control_percent = None,
                         sample_percent = None, threshold=None,
                         threshold_list = [], title="Neg Control vs Cell Lines 1,2,3"):
    names = list(samples.keys())
    samples_array = samples[names[0]].values.flatten()
    for i in names[1:]:
        np.concatenate([samples_array,samples[i].values.flatten()])
    samples = samples_array
        
    samples = skipZero(np.sort(samples))
    neg_cont= np.sort(neg_cont.values)
    
    plt.rc('axes', titlesize=25)
    plt.rc('axes', labelsize=15)
    plt.rc('xtick', labelsize=15)
    plt.rc('ytick', labelsize=15)
    
    fig = plt.figure(figsize=(12,8))
    plt.xscale('log')
    plt.hist(samples, alpha=.5,bins=np.logspace(np.log10(5), 10), label="Sample Data")
    plt.hist(neg_cont, alpha = .5, bins=np.logspace(np.log10(5), 10), label="Negative Control")
    
    plt.legend(loc='upper right')

    plt.title(title)
    for t in threshold_list:
        plt.axvline(x= t, linestyle='-', color="black")
        n_per = (len([x for x in neg_cont if x > t])/len(neg_cont)) * 100.0
        s_per = (len([x for x in samples if x > t])/len(samples)) *100.0
        print("{0:.2f}% control & {1:.2f}% sample are above {2}".format(n_per, s_per, t))
    if threshold:
        plt.axvline(x= threshold, linestyle='-', color="black")
        n_per = (len([x for x in neg_cont if x < threshold])/len(neg_cont)) * 100.0
        s_per = (len([x for x in samples if x < threshold])/len(samples)) *100.0
        print ("{0:.2f}% of neg cont is below solid line.".format(n_per))
        print ("{0:.2f}% of sample data is above the solid line.".format(100.0-s_per))
    if control_percent:
        threshold =  float(neg_cont[math.ceil(float(len(neg_cont))*(control_percent/100))])
        plt.axvline(x= threshold, linestyle='dashed', color="black")
        s_per = (len([x for x in samples if x < threshold])/len(samples)) * 100.0
        print ("{0:.2f}% of neg cont is below dashed line.".format(control_percent))
        print ("{0:.2f}% of sample data is above the dashed line.".format(100.0-s_per))
    if sample_percent:
        threshold =  float(samples[math.ceil(float(len(samples))*(sample_percent/100))])
        plt.axvline(x= threshold, linestyle='dotted', color="black")
        n_per = (len([x for x in neg_cont if x < threshold])/len(neg_cont)) * 100.0
        print ("{0:.2f}% of neg cont is below dotted line.".format(n_per))
        print ("{0:.2f}% of sample data is above the dotted line.".format(100.0-sample_percent))
    
    plt.xlabel("Raw Intensity Value")
    plt.ylabel("Number of Proteins")

    plt.show()

In [29]:
def graphed_types(samples, neg_cont, boost, control_percent = None,
                         sample_percent = None, threshold=None): 
    plt.rc('axes', titlesize=25)
    plt.rc('axes', labelsize=15)
    plt.rc('xtick', labelsize=15)
    plt.rc('ytick', labelsize=15)
    
    fig = plt.figure(figsize=(12,8))
    plt.xscale('log')

    kwargs = dict(histtype='stepfilled', alpha = .3, bins=np.logspace(np.log10(5), 10), edgecolor='black', linewidth=2)

    plt.hist(neg_cont, color = 'red',label="Negative Control", **kwargs)
    for i in samples:
        plt.hist(samples[i].values.flatten(), label=i, **kwargs)
    plt.hist(boost, color='purple', label="Boost", **kwargs)

    plt.legend(loc='upper right')

    plt.title("Neg Control vs Samples vs Boost")
    plt.xlabel("Raw Intensity Value")
    plt.ylabel("Number of Proteins")

    #fig.savefig('figures/NegSampBoost.png', dpi=300)
    
    names = list(samples.keys())
    samples_array = samples[names[0]].values.flatten()
    for i in names[1:]:
        np.concatenate([samples_array,samples[i].values.flatten()])
    samples = samples_array
        
    samples = skipZero(np.sort(samples))
    neg_cont= np.sort(neg_cont.values)
    
    if threshold:
        plt.axvline(x= threshold, linestyle='-', color="black")
        n_per = (len([x for x in neg_cont if x < threshold])/len(neg_cont)) * 100.0
        s_per = (len([x for x in samples if x < threshold])/len(samples)) *100.0
        b_per = (len([x for x in boost if x < threshold])/len(boost))*100.0
        print ("{0:.2f}% of neg cont is below solid line.".format(n_per))
        print ("{0:.2f}% of sample data is above the solid line.".format(100.0-s_per))
        print ("{0:.2f}% of boost data is above the solid line.".format(100.0-b_per))
    if control_percent:
        threshold =  float(neg_cont[math.ceil(float(len(neg_cont))*(control_percent/100))])
        plt.axvline(x= threshold, linestyle='dashed', color="black")
        s_per = (len([x for x in samples if x < threshold])/len(samples)) * 100.0
        b_per = (len([x for x in boost if x < threshold])/len(boost))*100.0
        print ("{0:.2f}% of neg cont is below dashed line.".format(control_percent))
        print ("{0:.2f}% of sample data is above the dashed line.".format(100.0-s_per))
        print ("{0:.2f}% of boost data is above the dashed line.".format(100.0-b_per))
    if sample_percent:
        threshold =  float(samples[math.ceil(float(len(samples))*(sample_percent/100))])
        plt.axvline(x= threshold, linestyle='dotted', color="black")
        n_per = (len([x for x in neg_cont if x < threshold])/len(neg_cont)) * 100.0
        b_per = (len([x for x in boost if x < threshold])/len(boost))*100.0
        print ("{0:.2f}% of neg cont is below dotted line.".format(n_per))
        print ("{0:.2f}% of sample data is above the dotted line.".format(100.0-sample_percent))
        print ("{0:.2f}% of boost data is above the dotted line.".format(100.0-b_per))
        
    
    plt.show()    

In [34]:
def zeros_by_type(samples, neg_cont, boost):
    n =len(skipZero(neg_cont))
    t = len(neg_cont)
    print ("Non-zeros in Neg. Control:\t{1:.2f}%\t({0} of {2})".format(n, n/t*100,t))
    
    for i in samples:
        l = samples[i].values.flatten()
        t = len(l)
        s = len(skipZero(l))
        percent = s/t*100
        print ("Non-zeros in {0}:\t{1:.2f}%\t({2} of {3})".format(i, percent, s, t))
    
    b =len(skipZero(boost))
    t = len(boost)
    print ("Non-zeros in boost:\t\t{1:.2f}%\t({0} of {2})".format(b, b/t*100, t))

In [25]:
def above_v(alist,list_name, min_threshold):
    p = (len([x for x in alist if x > min_threshold])/len(alist)*100)
    print ('{0:.2f} % of {1} falls above the threshold'.format(p, list_name))
    alist = skipZero(alist)
    p = (len([x for x in alist if x > min_threshold])/len(alist)*100)
    print ('{0:.2f} % of {1} without zeros falls above the threshold'.format(p, list_name))
    print()

def above_threshold(samples, neg_cont, boost, threshold):
    above_v(neg_cont, "Negative Control", threshold)
    for i in samples:
        above_v(samples[i].values.flatten(), i, threshold)
    above_v(boost, "Boost", threshold)